# Uploading data into PostgreSQL

**This notebook will load prepared training sets for each campaign section, containing processed features and their corresponding target variable, and append them to their respective PostgreSQL databases.**

In [6]:
# Load required libraries
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
from sklearn.externals import joblib

We'll be adding data into two distinct databases, `section1` and `section2`&mdash;one for each campaign section.

In [7]:
# Set Postgres credentials
db_name1 = 'section1'
db_name2 = 'section2'
usernm = 'redwan'
host = 'localhost'
port = '5432'
# pwd = ''

Next, let's create a SQLAlchemy engine for both database connections.

In [8]:
# Create engines for both databases
engine1 = create_engine(
    'postgresql://{}:{}@{}:{}/{}'.format(usernm, pwd, host, port, db_name1)
)

engine2 = create_engine(
    'postgresql://{}:{}@{}:{}/{}'.format(usernm, pwd, host, port, db_name2)
)

We'll create a database only if it doesn't yet exist.

In [9]:
# Create a new database for each section if it already does not exist
if not database_exists(engine1.url):
    create_database(engine1.url)

if not database_exists(engine2.url):
    create_database(engine2.url)

# Display whether the database exists
print(database_exists(engine1.url), database_exists(engine2.url) )

True True


Let's load the prepared training sets for each campaign section.

In [11]:
# Load DataFrames from pickle files
section1_df = joblib.load(
    'data/extracted_data/section1_extracted_data_20000-24558.pkl'
)

section2_df = joblib.load(
    'data/extracted_data/section2_extracted_data_20000-24558.pkl'
)

Next, let's upload and append the tables containing the training sets into their respective SQL databases.

In [12]:
# Append data into the corresponding SQL database
section1_df.to_sql(
    name=db_name1, 
    con=engine1,
    if_exists='append'
)

section2_df.to_sql(
    name=db_name2, 
    con=engine2,
    if_exists='append'
)

Finally, let's examine if the uploading was successful by querying the complete SQL database for a campaign section.

In [13]:
# Connect to a database
con1 = psycopg2.connect(
    database=db_name1, 
    host='localhost',
    user=usernm,
    password=pwd
)

# Define a SQL query to load a section
sql_query = """
SELECT * 
  FROM section1;
"""

# Perform SQL query and store results in a DataFrame
test_data_from_sql = pd.read_sql_query(sql_query, con1)

# Display the first five rows
test_data_from_sql.head()

,level_0,index,num_sents,num_words,num_all_caps,percent_all_caps,num_exclms,percent_exclms,num_apple_words,percent_apple_words,...,num_bolded,percent_bolded,name,category,hyperlink,currency,pledged,goal,location,funded
0,0,88389,15.0,223.0,0.0,0.000000,3.0,0.002317,0.0,0.000000,...,0.0,0.000000,Help me start my cottage industry ... Bakesale...,Small Batch,https://www.kickstarter.com/projects/138529431...,USD,0.0,10000.0,"Cape Coral, FL",False
1,1,190378,15.0,383.0,0.0,0.000000,4.0,0.003089,0.0,0.000000,...,0.0,0.000000,The Sock Who Lost His Mate at NY Children's Th...,Musical,https://www.kickstarter.com/projects/987315242...,USD,2600.0,7000.0,"Greenwich Village, Manhattan, NY",False
2,2,21028,15.0,259.0,17.0,0.013127,0.0,0.000000,1.0,0.000772,...,0.0,0.000000,The 4 Disciples,Comic Books,https://www.kickstarter.com/projects/the4disci...,USD,165.0,2200.0,"Rahway, NJ",False
3,3,154207,18.0,431.0,0.0,0.000000,3.0,0.002317,1.0,0.000772,...,0.0,0.000000,Every Precious Boy and Girl - Stories of Women...,Children's Books,https://www.kickstarter.com/projects/646886749...,USD,1836.0,1590.0,"Ann Arbor, MI",True
4,4,46223,23.0,283.0,2.0,0.001544,15.0,0.011583,0.0,0.000000,...,12.0,0.009266,Message in a Bottle Necklaces,Jewelry,https://www.kickstarter.com/projects/764561485...,USD,11.0,100.0,"Pasadena, CA",False


We can read in the data from the database. Let's see how large the new database is.

In [14]:
len(test_data_from_sql)

24558

We can also examine the database information.

In [15]:
test_data_from_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24558 entries, 0 to 24557
Data columns (total 29 columns):
level_0                    24558 non-null int64
index                      24558 non-null int64
num_sents                  24527 non-null float64
num_words                  24450 non-null float64
num_all_caps               24527 non-null float64
percent_all_caps           24450 non-null float64
num_exclms                 24527 non-null float64
percent_exclms             24450 non-null float64
num_apple_words            24527 non-null float64
percent_apple_words        24450 non-null float64
avg_words_per_sent         24447 non-null float64
num_paragraphs             24527 non-null float64
avg_sents_per_paragraph    24116 non-null float64
avg_words_per_paragraph    24116 non-null float64
num_images                 24527 non-null float64
num_videos                 24527 non-null float64
num_youtubes               24527 non-null float64
num_gifs                   24527 non-null flo